# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [1]:
! pip install syft

     |████████████████████████████████| 256kB 2.8MB/s 
     |████████████████████████████████| 81kB 24.5MB/s 
     |████████████████████████████████| 204kB 43.1MB/s 
     |████████████████████████████████| 389kB 39.8MB/s 
     |████████████████████████████████| 1.4MB 40.5MB/s 
     |████████████████████████████████| 256kB 43.9MB/s 
     |████████████████████████████████| 460kB 43.0MB/s 
     |████████████████████████████████| 51kB 18.4MB/s 
     |████████████████████████████████| 266kB 37.1MB/s 
     |████████████████████████████████| 122kB 42.5MB/s 
  Created wheel for zstd: filename=zstd-1.4.1.0-cp36-cp36m-linux_x86_64.whl size=1067085 sha256=963e0211885e84438234715571fa773ea5b04c54c8383592d0e0558c098fa92f
  Stored in directory: /root/.cache/pip/wheels/66/3f/ee/ac08c81af7c1b24a80c746df669ea3cb37542d27877d66ccf4
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44105 sha256=fc5f3188cbf568ca80cb61205970177e060a7d91308540877aec4a8a9982c6c1
  Stored in di

In [47]:
import syft as sy 
import torch as th 
hook = sy.TorchHook(th)
from torch import nn, optim

W0808 01:58:50.918555 140059899172736 hook.py:98] Torch was already hooked... skipping hooking process


In [0]:
# create some work 

bob = sy.VirtualWorker(hook, id = "bob")
alice = sy.VirtualWorker(hook, id = "alice")
secure_work = sy.VirtualWorker(hook, id = "secure_work")

In [49]:
bob.add_workers([alice, secure_work])
alice.add_workers([bob, secure_work])
secure_work.add_workers([alice, bob])

W0808 01:59:01.259849 140059899172736 base.py:646] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0808 01:59:01.262491 140059899172736 base.py:646] Worker secure_work already exists. Replacing old worker which could cause                     unexpected behavior
W0808 01:59:01.266088 140059899172736 base.py:646] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0808 01:59:01.267989 140059899172736 base.py:646] Worker secure_work already exists. Replacing old worker which could cause                     unexpected behavior
W0808 01:59:01.270416 140059899172736 base.py:646] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0808 01:59:01.271731 140059899172736 base.py:646] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_work #objects:0>

In [0]:
data = th.tensor([[1.,1], [0,1], [1,0], [0,0]], requires_grad = True)
target = th.tensor([[1.], [1],[0], [0]], requires_grad = True)

In [0]:
# get pointer to train data on each worker by 
# sending some trainingg data to the bob

bob_data = data[0:2].send(bob)
bob_targets =  target[0:2].send(bob)

In [0]:
alice_data = data[2:].send(alice)
alice_targets =  target[0:2].send(alice)

In [0]:
# model intalize A toy model 
model = nn.Linear(2, 1)

In [9]:
for iter in range ( 10):
  bobs_model = model.copy().send(bob)
  alice_model = model.copy().send(alice)

  bob_optim = optim.SGD(params= bobs_model.parameters(), lr = 0.1)
  alice_optim = optim.SGD(params= alice_model.parameters(), lr = 0.1)

  for i in range (10):
    bob_optim.zero_grad()
    bobs_pred = bobs_model(bob_data)
    bob_los = ((bobs_pred -bob_targets)**2 ).sum()
    bob_los.backward()
    bob_optim.step()
    bob_los = bob_los.get().data
    bob_los
    alice_optim.zero_grad()
    alice_pred = alice_model(alice_data)
    alice_los = ((alice_pred -alice_targets)**2 ).sum()
    alice_los.backward()
    alice_optim.step()
    alice_los = alice_los.get().data
    alice_los


  alice_model.move(secure_work)
  bobs_model.move(secure_work)
  
  model.weight.data = (((alice_model.weight.data + bobs_model.weight.data) / 2).get())
  model.bias.data = (((alice_model.bias.data + bobs_model.bias.data) / 2).get())
  secure_work.clear_objects()
  
  print("Bob:" + str(bob_los) + " Alice:" + str(alice_los))

Bob:tensor(0.0955) Alice:tensor(0.1619)
Bob:tensor(0.0227) Alice:tensor(0.0394)
Bob:tensor(0.0042) Alice:tensor(0.0100)
Bob:tensor(0.0006) Alice:tensor(0.0029)
Bob:tensor(6.2483e-05) Alice:tensor(0.0010)
Bob:tensor(4.0266e-07) Alice:tensor(0.0005)
Bob:tensor(4.9374e-06) Alice:tensor(0.0002)
Bob:tensor(9.9990e-06) Alice:tensor(0.0002)
Bob:tensor(1.0841e-05) Alice:tensor(0.0001)
Bob:tensor(9.6149e-06) Alice:tensor(7.1707e-05)


In [10]:
secure_work._objects

{}

# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [0]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [12]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [13]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [14]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [16]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [17]:
(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [0]:
x_share = (2,5,7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [0]:
Q = 23748629843768230486723


In [0]:
import random
def encrypt(x, n_share = 3):
 
  share = list()

  for i in range(n_share - 1):
     share.append(random.randint(0, Q))

  final_share = Q - (sum(share) % Q) + x 
  
  share.append(final_share)
  return tuple(share)

In [39]:
encrypt(x = 5, n_share= 10)

(14221890206917799707078,
 19496311619491905620106,
 4255354011884255794351,
 17549238763354485617412,
 7651189894495561739248,
 7906692280486448667406,
 12934453826315038018017,
 14251945976247096749211,
 4059424554710620208575,
 16416648084937940312216)

In [0]:
def dencrypt(shares):
  return sum(shares) % Q

In [41]:
dencrypt(encrypt(5))

5

In [0]:
def add(a, b):
  c = list()
  assert (len(a) == len(b))
  for i in range (len(a)):
    c.append((a[i] + b[i]) % Q)
  return tuple(c)

In [29]:
dencrypt(add(encrypt(5), encrypt(6)))

11

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [0]:
BASE=10
PRECISION=4

In [0]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [33]:
encode(3.5)

35000

In [34]:
decode(35000)

3.5

In [45]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(dencrypt(z))

7.8

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [0]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_work = secure_work.clear_objects()

In [0]:
x = th.tensor([1,2,3,4,5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [0]:
x = x.share(bob, alice, secure_work)

In [66]:
bob._objects

{51591291906: tensor([1876028943810375622, 2732447381892942038, 2163143366291919739,
         1875367143402587242,  203835981356664480])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [0]:
y = x + x

In [69]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> [PointerTensor | me:13505668009 -> bob:27888997414]
	-> [PointerTensor | me:24231432047 -> alice:15763553624]
	-> [PointerTensor | me:96342628289 -> secure_work:32282583245]
	*crypto provider: me*

In [70]:
bob._objects

{27888997414: tensor([3752057887620751244,  853208745358496172, 4326286732583839478,
         3750734286805174484,  407671962713328960]),
 51591291906: tensor([1876028943810375622, 2732447381892942038, 2163143366291919739,
         1875367143402587242,  203835981356664480])}

In [71]:
y.get()

tensor([ 2,  4,  6,  8, 10])

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [0]:
x = th.tensor([0.1,0.2,0.3])

In [84]:
x

tensor([0.1000, 0.2000, 0.3000])

In [85]:
x = x.fix_prec()
x

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300])

In [0]:
x = x.float_prec()

In [80]:
x

tensor([0.1000, 0.2000, 0.3000])

In [87]:
type(x.child)

syft.frameworks.torch.tensors.interpreters.precision.FixedPrecisionTensor

In [0]:
y = x + x

In [90]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

### Shared Fixed Precision

And of course, we can combine the two!

In [0]:
x = th.tensor([0.1, 0.2, 0.3])

In [0]:
x = x.fix_prec().share(bob, alice, secure_work)

In [0]:
y = x + x

In [95]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation